# W251 - Final Projects - Skeggox Team

* Parham Motameni 
* Mitchell Karchemsky
* Lynn Marciano

The majority of parameters can be configured, including some of the optimization techniques. Check out the configuration section for details.


In [1]:
%%capture
from pathlib import Path
data_path = 'grab.zip'
data_file = Path(data_path)
if data_file.is_file():
  print("Data file is already loaded.")
else :
  !pip install timm
  !pip install albumentations==0.4.6
  !pip install gdown
  !mkdir -p 'grab'
  !mkdir -p 'checkpoints'
  !gdown --id '1c8thCkdHJaoF4hLffi8ne-ULh4FPrPdy'
  !unzip -qq 'grab.zip'

In [2]:
import time
import os
from tqdm import tqdm_notebook as tqdm
#import tqdm.notebook import tqdm
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import random
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import DataLoader, Dataset
from torch.optim import lr_scheduler
import timm
import warnings
import torch.nn.functional as F
from torch.nn.modules.loss import _WeightedLoss
import torchvision.datasets as datasets
from timm.scheduler.cosine_lr import CosineLRScheduler
import shutil
from torch.cuda.amp import autocast, GradScaler
import torch.backends.cudnn as cudnn
import torchvision.models as models
import torch.nn as nn
import torchvision.transforms as transforms


warnings.filterwarnings('ignore')

# Configuration

The majority of parameters can be configured, including some of the optimization techniques.

The following optimizations can be used by setting proper configuration arguments
1. Learning Rate Schedular - Step Schedular or CosineLRScheduler
2. Different arch using mixnet_xl 

The following optimizations are applied 
1. Mixup
2. Label Smoothing
3. Scaler for Mixed Precision

In [3]:
class args:
  arch = 'mixnet_xl'
  # arch = 'efficientnet_b2' 
  batch_size = 20 #96
  beta = .1
  cudnn_benchmark = True
  cudnn_deterministic = True
  epochs = 100
  # epochs = 15
  folds = 8
  gpu = 0
  img_size = 224
  img_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
  img_std_RGB = [0.229, 0.224, 0.225]
  lr = 0.0001 # learning rate
  lr_min=0.00001 # min learning rate when using a LR Scheduler
  momentum = 0.9 # a setting for Optimizer
  warmup_epochs = 2 # a setting for Cosine LR Scheduler
  num_classes = 3 # number of categories 
  num_workers = 8 
  pretrained = False # determine if the weights of pre-build model should be used 
  print_freq = 50
  project_name = 'skeggox'
  resume = True # when it is True it will continue from last saved checkpoint
  save_checkpoint = True # save both last and best acc checkpoints
  save_checkpoint_per_epoch = True # save one file per epoch along best acc 
  seed = 1
  schedular_type = 'step' # option: 'cosine', 'step'(DEFAULT)
  start_epoch = 0
  test_batch_size = 128
  test_dir = 'grab/test'
  train_batch_size = 128
  train_dir = 'grab/train'
  warm = 4
  warmup_lr_init = 0.00001 # for Cosine LR
  weight_decay = 0.00001

In [4]:
class state:
  best_acc = 0
  start_epoch = 0

# Transformers

In [5]:
class CustomTransforms():
  def __init__(self):
    self.train_transform = CustomTransforms.create_train_transform()
    self.test_transform = CustomTransforms.create_test_transform()

  @staticmethod
  def create_train_transform():
     return A.Compose([
        A.Resize(args.img_size, args.img_size),
        A.HorizontalFlip(p=0.5),
        A.Transpose(p=0.5),
        ToTensorV2(),
        ])
    # return A.Compose([
    #   transforms.RandomResizedCrop(args.img_size),
    #   transforms.RandomHorizontalFlip(),
    #   transforms.ToTensor(),
    #   transforms.Normalize(args.img_mean_RGB, args.img_std_RGB),
    # ])
  @staticmethod
  def create_test_transform():
    return A.Compose([
        ToTensorV2(),
    ])
    # return A.Compose([
    #     transforms.Resize(args.img_size),  
    #     transforms.CenterCrop(args.img_size),
    #     transforms.ToTensor(),
    #     transforms.Normalize(args.img_mean_RGB, args.img_std_RGB),
    # ])

# Load Dataset

In [6]:
class SkeggoxDataset(Dataset):
    def __init__(self, df, mode, img_dir, transform=None):
        self.data = df
        self.img_dir = img_dir
        self.transform = transform
        self.mode = mode

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):        
        fname = self.data.iloc[idx]['filename']
        img_path = (f'{self.img_dir}/{fname}')
        image = cv2.imread(img_path)
        if self.transform is not None:
          image = self.transform(image = image)['image']
        image = image.float() / 255.
        label = -1 if self.mode=='test' else self.data.iloc[idx]['label']
        
        return image, label

In [7]:
class DataCollections():
  def __init__(self):
    self.load_datasets()
    self.build_dataloaders()

  def load_datasets(self):
    alldf = pd.read_csv(f'grab/train.csv')
    # if you want to change the file path 
    # alldf['filename'] = 'train/' + alldf['filename'] + '.jpeg'

    # Split the training dataset into a training and a validation
    valdf = alldf[::args.folds]
    trndf = alldf[~alldf.filename.isin(valdf.filename)]

    # Load our test data
    tstdf = pd.read_csv(f'grab/test.csv')
    # if you want to change the file path 
    # tstdf['filename'] = 'test/' + tstdf['filename'] + '.jpeg'

    print(f'File shapes -- train : {trndf.shape}, valid : {valdf.shape}, test : {tstdf.shape}')

    custom_transforms = CustomTransforms()

    self.train_dataset = SkeggoxDataset(trndf,  'train', args.train_dir,  custom_transforms.train_transform)
    self.validate_dataset = SkeggoxDataset(valdf, 'train', args.train_dir, custom_transforms.train_transform)
    self.test_dataset = SkeggoxDataset(tstdf, 'test', args.test_dir, custom_transforms.test_transform)

    # return train_dataset, validate_dataset, test_dataset

  def build_dataloaders(self):

    loaderargs = {'num_workers' : args.num_workers, 'batch_size':args.batch_size,
                  'pin_memory': False, 'drop_last': False}
  
    self.train_dataloader = DataLoader(self.train_dataset, shuffle = True, **loaderargs)
    self.validate_dataloader = DataLoader(self.validate_dataset, shuffle = False, **loaderargs)
    self.test_dataloader = DataLoader(self.test_dataset, shuffle = False, **loaderargs)

In [8]:
# TODO: This is just for test, remove it for final submisson
def test_data_collections():
  # Test the dataset
  cls = DataCollections()
  img, label = next(iter(cls.train_dataset))
  imgviz = (img * 255).transpose(0, 2).numpy().astype(np.uint8)
  return Image.fromarray(imgviz)
# img = test_data_collections()
# img

# Save and Resume 

In [9]:
def save_checkpoint(state, is_best):
  if args.save_checkpoint or args.save_checkpoint_per_epoch:
    folder = 'checkpoints/'
    filename = f'{folder}checkpoint.pth.tar'
    torch.save(state,  filename)
    if args.save_checkpoint_per_epoch:
      shutil.copyfile(filename, f'{folder}checkpoint_{state["epoch"]}.pth.tar')
    if is_best:
      shutil.copyfile(filename, f'{folder}model_best.pth.tar')

In [10]:
def resume(sk_model, filename = 'checkpoints/checkpoint.pth.tar' ):
  if args.resume:
    if os.path.isfile(filename):
      print(f'=> loading checkpoint: {filename}')
      checkpoint = torch.load(filename, map_location=sk_model.device)
      state.start_epoch = checkpoint['epoch']
      state.best_acc = checkpoint['best_acc']
      # state.best_acc = best_acc.to(GPU)
      sk_model.model.load_state_dict(checkpoint['state_dict'])
      sk_model.optimizer.load_state_dict(checkpoint['optimizer'])
      print(f'=> loaded checkpoint {filename}',
            f'epoch {state.start_epoch}')
    else:
        print(f'=> no checkpoint found at {filename}')

# Set Model

In [11]:
class SkeggoxMetrics:
  def __init__(self):
    self.lr_list = []

In [12]:
# Label Smoothing for Cross Entropy Loss fucntion
# From https://github.com/pytorch/pytorch/issues/7455#issuecomment-759175034
class LabelSmoothCrossEntropyLoss(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth_one_hot(targets: torch.Tensor, n_classes: int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = torch.empty(size=(targets.size(0), n_classes),
                                  device=targets.device) \
                .fill_(smoothing / (n_classes - 1)) \
                .scatter_(1, targets.data.unsqueeze(1), 1. - smoothing)
        return targets

    def forward(self, inputs, targets):
        targets = LabelSmoothCrossEntropyLoss._smooth_one_hot(targets, inputs.size(-1),
                                                              self.smoothing)
        lsm = F.log_softmax(inputs, -1)

        if self.weight is not None:
            lsm = lsm * self.weight.unsqueeze(0)

        loss = -(targets * lsm).sum(-1)

        if self.reduction == 'sum':
            loss = loss.sum()
        elif self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [13]:
class SkeggoxModel():
  def __init__(self, device, arch, num_steps, pretrained = False):
    self.device = device
    self.pretrained = pretrained
    self.arch = arch
    self.num_steps = num_steps
    self.metrics = SkeggoxMetrics()

    self.create_model()
    self.create_criterion()
    self.create_optimizer()
    self.create_scheduler()
    # define scaler for Mixed Precision
    self.scaler = GradScaler()
    
  
  def create_model(self):
    # TODO you can add custom model instead of using pre-built model here
    model = timm.create_model(self.arch, pretrained = self.pretrained)
    # # model = models.__dict__[args.arch]()
    # inf = model.fc.in_features
    # model.fc = nn.Linear(inf, args.num_classes)
    # model.cuda(args.gpu)
    self.model = model.to(self.device)

  def create_criterion(self):
    ## Non-smoothing version
    # self.criterion = torch.nn.CrossEntropyLoss()
    ## the smoothing version
    self.criterion = LabelSmoothCrossEntropyLoss(smoothing=0.3)
  
  def create_optimizer(self):
    # Adam Optimizer
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=args.lr)
    # SGD Optimizer
    # self.optimizer =  torch.optim.SGD(self.model.parameters(), 
    #                        args.lr,
    #                        momentum=args.momentum,
    #                        weight_decay=args.weight_decay)

  def create_scheduler(self):
    if args.schedular_type == 'cosine':
      # Cosine LR
      steps = self.num_steps
      self.scheduler = CosineLRScheduler(
              self.optimizer,
              t_initial = steps * args.epochs + 1,
              lr_min = args.lr_min,
              warmup_lr_init = args.warmup_lr_init,
              warmup_t = steps * args.warmup_epochs + 1)
    else:
      # step LR
      self.scheduler = lr_scheduler.StepLR(self.optimizer, step_size=10)

# Train and Validate

In [14]:
def train(sk_model, dl, global_step):
  print(f'Epoch {epoch}/{args.epochs - 1}')
  device = sk_model.device
  model = sk_model.model
  metrics = sk_model.metrics
  optimizer = sk_model.optimizer
  scaler = sk_model.scaler
  sk_model.scheduler.step(global_step) if args.schedular_type == 'cosine' \
      else sk_model.scheduler.step()
 
  model.train()
  running_loss = 0.0
  train_d = dl.train_dataloader
  tk0 = tqdm(train_d, total=int(len(train_d)))
  for step, batch in enumerate(tk0):
      inputs = batch[0].to(device, dtype=torch.float)
      labels = batch[1].to(device).long()

      ## Mixup
      # Get out a random value form a distribution    
      lam = np.random.beta(args.beta, args.beta)
      rand_index = torch.randperm(inputs.size()[0]).to(device) # make an index which reorders the batch

      # Reorder the labels
      labels_a = labels
      labels_b = labels[rand_index]

      # Partially mixup up the batch
      inputs_mixed = lam * inputs + (1 - lam) * inputs[rand_index]

      optimizer.zero_grad()

      # Runs the forward pass with autocasting.
      with autocast():
          # using mixed up input for model
          outputs = model(inputs_mixed)            
          # Partial loss against original labels, partial loss against mixed up labels
          loss = sk_model.criterion(outputs, labels_a) * lam + \
              sk_model.criterion(outputs, labels_b) * (1. - lam)
      

      # outputs = model(inputs)
      # loss = sk_model.criterion(outputs, labels)

      # Scaler
      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update()

      # non-scaler
      # loss.backward()
      # optimizer.step()

      running_loss += loss.item()
      tk0.set_postfix(train_loss=(running_loss / (step+1)))

      global_step += 1
      # print(optimizer.param_groups[0]['lr'])
      metrics.lr_list.append(optimizer.param_groups[0]['lr']) 


In [15]:
def validate(sk_model, dl):
  valpreds = []
  device = sk_model.device
  model = sk_model.model
  model.eval()
  running_loss = 0.0
  val_d = dl.validate_dataloader
  tkval = tqdm(val_d, total=int(len(val_d)))
  for step, batch in enumerate(tkval):
      inputs = batch[0].to(device, dtype=torch.float)
      labels = batch[1].to(device).long()
      with torch.no_grad():
          outputs = model(inputs)
          loss = sk_model.criterion(outputs, labels)
      valpreds.append(outputs)
      running_loss += loss.item()
      tkval.set_postfix(valid_loss=(running_loss / (step+1)))
  preds = torch.cat(valpreds).argmax(1).detach().cpu().numpy()
  acc = (dl.validate_dataset.data.label.values == preds).mean()
  print(f'Valid accuracy {acc:.4f}')
  
  is_best = acc > state.best_acc
  state.best_acc = max(acc, state.best_acc)

  save_checkpoint({
      'epoch': epoch + 1,
      'step': step + 1,
      'arch': args.arch,
      'state_dict': model.state_dict(),
      'best_acc':  state.best_acc,
      'optimizer' : sk_model.optimizer.state_dict(),
  }, is_best)

# Run

In [ ]:
since = time.time()
dl = DataCollections()
device = torch.device("cuda:0")
sk_model = SkeggoxModel(device, args.arch, len(dl.train_dataloader), pretrained = True)
global_step = 0
state.best_acc = 0
resume(sk_model)
for epoch in range(state.start_epoch, args.epochs):
  train(sk_model, dl, global_step)
  validate(sk_model, dl)

File shapes -- train : (956, 2), valid : (137, 2), test : (1093, 2)
=> no checkpoint found at checkpoints/checkpoint.pth.tar
Epoch 0/99


  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Valid accuracy 0.6058
Epoch 1/99


  0%|          | 0/48 [00:00<?, ?it/s]

...

Valid accuracy 0.7372
Epoch 4/99


  0%|          | 0/48 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

Valid accuracy 0.8029
Epoch 5/99


  0%|          | 0/48 [00:00<?, ?it/s]